In [ ]:
"""Task:
You have an nxn array consisting out of 0s and 1s. The 1s represent land while the 0s represent water.
A central 1 that is horizontally or vertically connected to the edge of the array is referred as mainland.
Central 1s that has no horizontally or vertically connection to the edge of the array are called Islands.

The Challenge will be to replace all the islands with 0.

Example Input:
 [0 0 1 0 0 1 0 1 1 0]
 [0 0 1 0 0 1 1 1 1 1]
 [1 0 1 1 0 1 1 1 0 1]
 [0 0 1 1 1 1 1 1 0 1]
 [1 1 0 0 0 1 1 1 0 0]
 [0 1 0 1 1 0 0 0 0 1]
 [1 0 0 0 0 1 0 1 1 1]
 [0 1 0 1 1 0 1 1 1 1]
 [0 1 1 1 0 1 0 0 0 1]
 [1 0 1 0 0 0 0 0 0 1]

 Example Output:
 [0 0 1 0 0 1 0 1 1 0]
 [0 0 1 0 0 1 1 1 1 1]
 [1 0 1 1 0 1 1 1 0 1]
 [0 0 1 1 1 1 1 1 0 1]
 [1 1 0 0 0 1 1 1 0 0]
 [0 1 0 0 0 0 0 0 0 1]
 [1 0 0 0 0 0 0 1 1 1]
 [0 0 0 1 1 0 1 1 1 1]
 [0 0 1 1 0 0 0 0 0 1]
 [1 0 1 0 0 0 0 0 0 1]

 Removed 1:
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 1 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]

"""


In [72]:
import numpy as np
import pandas as pd

In [169]:
# Create the map as a DataFrame
array = np.random.randint(2, size=(10, 10))
map = pd.DataFrame(array)
print(map)

   0  1  2  3  4  5  6  7  8  9
0  1  0  1  0  1  1  1  1  0  1
1  0  1  0  1  0  1  1  0  1  1
2  0  0  0  0  0  0  1  1  0  1
3  0  1  1  0  1  1  0  0  1  1
4  1  0  0  1  0  1  1  0  0  0
5  1  0  1  0  0  1  1  0  1  0
6  0  1  0  0  0  1  1  0  1  1
7  1  0  1  1  0  0  1  0  0  0
8  1  1  0  1  1  1  0  0  0  1
9  1  1  1  0  1  0  1  1  1  1


In [170]:
# Create Row-Column-Coordinates for each Cell and transpose the table
map_table = map
map_table['Row']=map_table.index
map_table=map_table.melt(id_vars='Row',var_name='Column')
print(map_table)

    Row Column  value
0     0      0      1
1     1      0      0
2     2      0      0
3     3      0      0
4     4      0      1
..  ...    ...    ...
95    5      9      0
96    6      9      1
97    7      9      0
98    8      9      1
99    9      9      1

[100 rows x 3 columns]


In [177]:
# Create several DataFrame objects:
# - land: Contains all cells with value 1
# - mainland: Contains all cells with value 1 that are at the edge
# - potential_islands: Contains all cells with value 1 that do not have an immediate connection to the edge
# - Start: A copy of mainland used in the while loop
# - Already_Tested: DataFrame containing all cells that have already been tested, regardless of whether they are connected to a potential island or not
land = map_table[map_table['value']==1]
mainland = land[(land['Row']==0) | (land['Column']==0) | (land['Row']==land['Row'].max()) | (land['Column']==land['Column'].max())  ]

potential_islands = land[(land['Row']!=0) & (land['Column']!=0) & (land['Row']!=land['Row'].max()) & (land['Column']!=land['Column'].max())  ]
Start = mainland.copy()
Already_tested = pd.DataFrame()

In [181]:
""" Idea:
    1. Identify all horizontal and vertical neighbor cells of the starting cells.
    2. (Filter out all cells that have already been checked in previous iterations - not important for the first round.)
    3. Filter out all cells that are not included in the potential_island DataFrame.
      --- The remaining cells are still part of the potential island and not in the mainland. ---
    4. Remove these cells from the "potential_islands" status and add them to mainland.
    5. Repeat this process with the newly added islands.

    
    """

while True:

    # 1. Identify all horizontal and vertical neighbor cells of the starting cells
    Radd1 = Start.copy()
    Rsub1 = Start.copy()
    Cadd1 = Start.copy()
    Csub1 = Start.copy()

    Radd1['Row']=Radd1['Row']+1
    Rsub1['Row']=Rsub1['Row']-1
    Cadd1['Column']=Cadd1['Column']+1
    Csub1['Column']=Csub1['Column']-1

    examine_table= pd.concat([Radd1,Rsub1,Cadd1,Csub1]).drop_duplicates()

    # 2. Filter out all cells that have already been checked in previous iterations
    examine_table=pd.concat([examine_table,Already_tested]).drop_duplicates(keep=False)

    # (2 b) Update the Already_Tested DataFrame
    Already_tested=pd.concat([examine_table,Already_tested]).drop_duplicates()

    # 3. Filter out all cells that are not included in the potential_island DataFrame.
    examine_table = pd.merge(potential_islands,examine_table,left_on=['Row','Column'],right_on=['Row','Column'],how='inner').iloc[:,:3]
    examine_table=examine_table.rename(columns={'value_x':'value'})
    
    # 4. Remove these cells from the "potential_islands" status and add them to mainland
    potential_islands=pd.concat([potential_islands,examine_table]).drop_duplicates(keep=False)
    mainland=pd.concat([mainland,examine_table]).drop_duplicates()

    # 5. Repeat this process with the newly added islands. Loop ends if nothing new has been found.
    if examine_table.empty:
        break
    Start = examine_table

# Replace the Values of potential_islands with 0 - They are actually no potential islands anymore.
potential_islands['value']=0

# combine and restructure the DataFrame so we have an array again.
result = pd.concat([mainland,potential_islands,map_table])
result = pd.concat([mainland,potential_islands,map_table]).drop_duplicates(subset=['Row','Column'],keep='first')
result = result.sort_values(by='Row')
result = result.pivot(index='Row', columns='Column', values='value')
print(result)


Column  0  1  2  3  4  5  6  7  8  9
Row                                 
0       1  0  1  0  1  1  1  1  0  1
1       0  0  0  0  0  1  1  0  1  1
2       0  0  0  0  0  0  1  1  0  1
3       0  0  0  0  0  0  0  0  1  1
4       1  0  0  0  0  0  0  0  0  0
5       1  0  0  0  0  0  0  0  1  0
6       0  0  0  0  0  0  0  0  1  1
7       1  0  1  1  0  0  0  0  0  0
8       1  1  0  1  1  1  0  0  0  1
9       1  1  1  0  1  0  1  1  1  1
